Notebook written by [Zhedong Zheng](https://github.com/zhedongzheng)

<img src="img/convlstm.png" width="400">

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
VOCAB_SIZE = 5000
MAX_LEN = 400
CONV_LEN = 10
BATCH_SIZE = 32
EMBED_DIM = 50
FILTERS = 100
CLIP_NORM = 5.0
N_CLASS = 2
N_EPOCH = 2
LR = {'start': 5e-3, 'end': 5e-4, 'steps': 1500}

In [3]:
def clip_grads(loss):
    params = tf.trainable_variables()
    grads = tf.gradients(loss, params)
    clipped_grads, _ = tf.clip_by_global_norm(grads, CLIP_NORM)
    return zip(clipped_grads, params)


def forward(x, mode):
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    batch_sz = tf.shape(x)[0]
    seq_len = tf.count_nonzero(x, 1) // CONV_LEN + 1
    
    x = tf.contrib.layers.embed_sequence(x, VOCAB_SIZE, EMBED_DIM)
    x = tf.layers.dropout(x, 0.2, training=is_training)
    x = tf.reshape(x, [batch_sz, MAX_LEN//CONV_LEN, CONV_LEN, EMBED_DIM])
    
    cell = tf.contrib.rnn.ConvLSTMCell(conv_ndims = 1,
                                       input_shape = [CONV_LEN, EMBED_DIM],
                                       output_channels = FILTERS,
                                       kernel_shape = [3])
    x = tf.nn.dynamic_rnn(cell, x, seq_len, dtype=tf.float32)[1].h
    x = tf.reduce_max(x, 1)
    
    x = tf.layers.dropout(x, 0.2, training=is_training)
    x = tf.layers.dense(x, FILTERS, tf.nn.relu)
    logits = tf.layers.dense(x, N_CLASS)
    return logits


def model_fn(features, labels, mode):
    logits = forward(features, mode)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        preds = tf.argmax(logits, -1)
        return tf.estimator.EstimatorSpec(mode, predictions=preds)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_global_step()

        lr_op = tf.train.exponential_decay(
            LR['start'], global_step, LR['steps'], LR['end']/LR['start'])

        loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

        train_op = tf.train.AdamOptimizer(lr_op).apply_gradients(
            clip_grads(loss_op), global_step=global_step)

        lth = tf.train.LoggingTensorHook({'lr': lr_op}, every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss_op, train_op=train_op, training_hooks=[lth])

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=VOCAB_SIZE)
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, MAX_LEN, padding='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, MAX_LEN, padding='post')

estimator = tf.estimator.Estimator(model_fn)

for _ in range(N_EPOCH):
    estimator.train(tf.estimator.inputs.numpy_input_fn(
        x = X_train, y = y_train,
        batch_size = BATCH_SIZE,
        shuffle = True))
    y_pred = np.fromiter(estimator.predict(tf.estimator.inputs.numpy_input_fn(
        x = X_test,
        batch_size = BATCH_SIZE,
        shuffle = False)), np.int32)
    print("\nValidation Accuracy: %.4f\n" % (y_pred==y_test).mean())

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpr4_s4__h', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x114c7b518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/sx/